In [ ]:
import os
import pandas as pd
import subprocess
import json
import requests
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import geopandas as gpd
from shapely.geometry import Point, LineString
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from tabulate import tabulate

import pyproj
import math

from transformers import AutoModel
from huggingface_hub import hf_hub_download
from ultralytics import YOLO

## ResNet50


In [ ]:
# 1)EXIF 
def exiftool_paring(failitee):
    try:
        cmd = ["exiftool", "-j", "-n", failitee]
        tulemus = subprocess.check_output(cmd)
        data = json.loads(tulemus)
        if data and isinstance(data, list):
            return data[0]
        return {}
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print("viga exiftool:", e)
        return {}

def loe_exif_info(droonifail):
    info = {
        'lat': None, 
        'lon': None, 
        'alt': None,
        'fov': None,
        'gimbal_yaw': None, 
        'gimbal_pitch': None, 
        'gimbal_roll': None,
        'flight_yaw': None,
        'flight_pitch': None,
        'flight_roll': None
    }
    meta = exiftool_paring(droonifail)
    if not meta:
        print("EXIF puudub või exiftool error:", droonifail)
        return info

    lat = meta.get('GPSLatitude')
    lon = meta.get('GPSLongitude')
    alt = meta.get('GPSAltitude')
    if lat is not None: info['lat'] = float(lat)
    if lon is not None: info['lon'] = float(lon)
    if alt is not None: info['alt'] = float(alt)

    fv = meta.get('Field Of View')
    if fv is not None:
        try:
            info['fov'] = float(fv)
        except:
            pass

    gy = meta.get('GimbalYawDegree')
    gp = meta.get('GimbalPitchDegree')
    gr = meta.get('GimbalRollDegree')
    fy = meta.get('FlightYawDegree')
    fp = meta.get('FlightPitchDegree')
    fr = meta.get('FlightRollDegree')

    if gy is not None: info['gimbal_yaw'] = float(gy)
    if gp is not None: info['gimbal_pitch'] = float(gp)
    if gr is not None: info['gimbal_roll'] = float(gr)
    if fy is not None: info['flight_yaw'] = float(fy)
    if fp is not None: info['flight_pitch'] = float(fp)
    if fr is not None: info['flight_roll'] = float(fr)

    return info

#2)lat/lon -> L-EST (EPSG:3301)
def proj_3301(lat, lon):
    tr = pyproj.Transformer.from_crs("EPSG:4326","EPSG:3301", always_xy=True)
    x,y = tr.transform(lon, lat)
    return x,y


#3)Maa-ameti WMS hankimine
def wms_maaamet_taust(bbox, width=1024, height=1024, kiht="EESTIFOTO"):
    ymin,xmin,ymax,xmax = bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin

    bbox_str= f"{xmin},{ymin},{xmax},{ymax}"

    wms_url= "https://kaart.maaamet.ee/wms/fotokaart"
    params= {
        "service":"WMS",
        "request":"GetMap",
        "version":"1.3.0",
        "layers":kiht,
        "styles":"",
        "crs":"EPSG:3301",
        "bbox": bbox_str,
        "width": str(width),
        "height": str(height),
        "format":"image/png",
        "transparent":"FALSE"
    }
    print("WMS parameetrid:", params)
    resp= requests.get(wms_url, params=params)
    resp.raise_for_status()
    ctype= resp.headers.get("Content-Type","")
    if not ctype.startswith("image/png"):
        print("Ei saa PNG, vaid:", ctype)
        return None
    
    pil_img= Image.open(BytesIO(resp.content)).convert("RGB")
    arr= np.array(pil_img)
    mean_val= arr.mean()
    print(f"WMS pildi keskmine={mean_val:.1f}")
    if mean_val>250:
        print("Hoiatus: pilt valge => ilmselt ei kata ala. Mean>250.")
    return pil_img

#4)Mudeli laadimine = DINOv2_base
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kasutan seadet:", device)

resnet_model = models.resnet50(pretrained=True)
resnet_model = nn.Sequential(*list(resnet_model.children())[:7]) 
resnet_model = resnet_model.eval().to(device)
print("Laetud ResNet50 mudel (7 kihti).")

class ResNetFeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        feats = self.backbone(x)  # shape (B, C, H, W)
        return feats

feat_extractor = ResNetFeatureExtractor(resnet_model).to(device)

map_transform = T.Compose([
    T.Resize((1024, 1024)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
drone_transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

@torch.no_grad()
def extrakti_omadused(pil_img, transform):
    x = transform(pil_img).unsqueeze(0).to(device)
    feats = feat_extractor(x)
    return feats

#5)Koosinus sarnasus => heatmap
def compute_heatmap_ncc(mapF, droneF):
    cross_term = F.conv2d(mapF, droneF, bias=None, stride=1)
    norm_drone = torch.sqrt(torch.sum(droneF**2))

    map_sq = mapF * mapF
    ones_ker = torch.ones_like(droneF)
    sum_map_sq = F.conv2d(map_sq, ones_ker, bias=None, stride=1)
    norm_map = torch.sqrt(sum_map_sq + 1e-8)

    ncc = cross_term / (norm_drone * norm_map + 1e-8)
    heatmap = ncc.squeeze(0).squeeze(0).cpu().numpy()
    return heatmap

#6)arvuta_bbox_fov => coverage ×4
def arvuta_bbox_fov(xLe,yLe, alt_sea, ground_offset=40.0, fov_deg=70.0):
    if alt_sea is None:
        alt_p=200.
    else:
        alt_p= alt_sea-ground_offset
        if alt_p<0:
            alt_p=50.
    rad= math.radians(fov_deg)
    laius= 2.0 * alt_p * math.tan(rad/2.0)
    laius = laius * 4
    pool= laius/2.
    return [xLe-pool, yLe-pool, xLe+pool, yLe+pool]

#7)Droonipildi YAW pööramine (drone image rotate yaw)
def paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=None):
    if yaw is None and flight_yaw is None:
        return droonPil

    if pitch is not None and abs(pitch + 90) < 10:
        angle = 360.0 - yaw if yaw is not None else 0.0
        if flight_yaw is not None:
            yaw_diff = abs((yaw - flight_yaw + 360) % 360)
            if 160 <= yaw_diff <= 200:  
                print(f"Yaw-flip tuvastatud (gimbal: {yaw:.1f}, flight: {flight_yaw:.1f}) => flip +180°")
                angle = (angle + 180) % 360
        print(f"Keeran droonipilti {angle:.1f} kraadi (nadir, gimbal yaw)")
        return droonPil.rotate(angle, expand=True)

    elif flight_yaw is not None:
        angle = 360.0 - flight_yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (flight yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    elif yaw is not None:
        angle = 360.0 - yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (fallback gimbal yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    return droonPil

# 8)pixel2lest => (col,row)->(x,y)
def pixel2lest(bbox, w, h, col, row):
    xmin, ymin, xmax, ymax= bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin
    x= xmin + (xmax - xmin)*(col/ float(w))
    y= ymax - (ymax - ymin)*(row/ float(h))
    return x,y

#9)Drooni teekonna ehitamine (building drone trajectory)
def drooni_teekond_gdf(flight_data):
    geometry=[]
    ajad=[]
    for d in flight_data:
        geometry.append(Point(d['x'], d['y']))
        ajad.append(d['aeg'])
    gdf= gpd.GeoDataFrame({'aeg': ajad}, geometry=geometry, crs="EPSG:3301")
    return gdf

#10)Drooni teekond (drone trajectory)
def joonista_drooni_teekond(gdf, margin=2000):
    minx, miny, maxx, maxy= gdf.total_bounds
    minx-= margin
    miny-= margin
    maxx+= margin
    maxy+= margin

    bbox= [minx, miny, maxx, maxy]
    orto= wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto is None:
        print("Ei saanud WMS => skip.")
        return
    
    arr= np.array(orto)
    fig, ax= plt.subplots(figsize=(10,10))
    ax.imshow(arr, extent=[minx, maxx, miny, maxy], origin='lower')
    gdf.plot(ax=ax, marker='o', color='red', markersize=50, label='Drooni punktid')

    if len(gdf)>1:
        line_geom= LineString(gdf.geometry.values)
        line_gdf= gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:3301")
        line_gdf.plot(ax=ax, color='blue', linewidth=2, label='Drooni teekond')

    for idx, row in gdf.iterrows():
        x_pt= row.geometry.x
        y_pt= row.geometry.y
        ax.text(x_pt, y_pt, str(idx+1), color='white', fontsize=12,
                ha='center', va='center', fontweight='bold')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_title("Drooni teekond (zoomitud, põhjasuunaline)")
    ax.legend()
    plt.show()

#Nihke arvutamine (offset calculation)
offsetid = []
kiirused = []
def arvuta_offset(x_est, y_est, x_gt, y_gt):
    dist = math.sqrt((x_est - x_gt)**2 + (y_est - y_gt)**2)  #Meetrites (in meters)
    return dist

#11)Täielik koodi implementeerimine (full pipeline)
def full_pipeline(kaust, ground_offset=40.0):
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Ei leidu droonifotosid:", kaust)
        return

    flight_data = []

    for fn in failid:
        path = os.path.join(kaust, fn)
        print("\nFail:", path)
        info = loe_exif_info(path)
        lat, lon = info['lat'], info['lon']
        alt = info['alt']
        fov = info['fov'] if info['fov'] else 70
        yaw = info['gimbal_yaw']
        pitch = info['gimbal_pitch']
        roll = info['gimbal_roll']

        if lat is None or lon is None:
            print("Puudub lat/lon => skip.")
            continue

        #1)Katvusala (4x katvus) (bounding box)
        xLe, yLe = proj_3301(lat, lon)
        bbox = arvuta_bbox_fov(xLe, yLe, alt, ground_offset, fov)
        print("=> BBox LEST:", bbox)

        #2)Hangin kaardi (1024×1024, 4x ülekate) (Fetching the map)
        orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
        if orto_pil is None:
            print("Ei saa WMS => skip.")
            continue

        #3)Lae droonipilt, keera ja muuda suurust (256×256) (loading drone image, rotate and resize)
        droonPil = Image.open(path).convert("RGB")
        droonPil = paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=info.get('flight_yaw'))
        droonPil = droonPil.resize((256, 256), Image.BILINEAR)

        #Näita droonipilti ja kaarti kõrvuti (showing the drone picture and map side by side)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
        ax1.imshow(orto_pil)
        ax1.set_title("Kaart => 1024×1024 bounding box (4x ülekate)")
        ax2.imshow(droonPil)
        ax2.set_title("Droon => 256×256")
        plt.show()

        algusaeg = time.time()

        #4)Võtan tunnused välja (shape => (1, C, Hf, Wf)) (Extracting features)
        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droonPil, drone_transform)

        #5)Arvutan heatmapi-slaidiv konvolutsioon (Calculating heatmap-sliding convolution)
        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]
        print(f"Parim simulatsioon => {best_val:.4f}, row={best_i}, col={best_j}")

        loppaeg = time.time()
        kestvus = loppaeg - algusaeg 
        kiirused.append(kestvus)

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='sarnasus (similarity)')
        plt.title(f"{fn}: max={best_val:.4f}")
        plt.show()

        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        #Nihke arvutamine tegeliku ja väljund punkti vahel (offset calculating of real and returned red dot)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = arvuta_offset(x_hit, y_hit, x_gt, y_gt)
        offsetid.append(offset)

    if not flight_data:
        print("Pole lennuandmeid -> (exit)")
        return

    gdf = drooni_teekond_gdf(flight_data)
    joonista_drooni_teekond(gdf, margin=2000)

    offsetid_np = np.array(offsetid)
    kiirused_np = np.array(kiirused)

    print("\n==== KOKKUVÕTE ====")
    print(f"Piltide arv: {len(offsetid)}")
    print(f"Keskmine offset: {offsetid_np.mean():.2f} m")
    print(f"Min offset: {offsetid_np.min():.2f} m")
    print(f"Max offset: {offsetid_np.max():.2f} m")
    print(f"Standardhälve: {offsetid_np.std():.2f} m")
    print(f"Keskmine heatmapi genereerimise aeg: {kiirused_np.mean()*1000:.1f} ms")
    print(f"Minimaalne aeg: {kiirused_np.min()*1000:.1f} ms")
    print(f"Maksimaalne aeg: {kiirused_np.max()*1000:.1f} ms")

    #tabelina
    tabel = pd.DataFrame({
        'Fail': failid,
        'Offset_m': offsetid_np,
        'Aeg_ms': kiirused_np * 1000
    })
    print("\nTABEL:")
    print(tabel)

def naide_kasutus():
    kaust = "/Users/gnepste/Desktop/Andmeteadus (KOOL)/MAGISTRITÖÖ/droonifotod"
    full_pipeline(kaust, ground_offset=40)

naide_kasutus()


## DINOv2

### DINOv2-base

In [ ]:
# 1)EXIF metaandmete hankimine (EXIF metadata)
def exiftool_paring(failitee):
    try:
        cmd = ["exiftool", "-j", "-n", failitee]
        tulemus = subprocess.check_output(cmd)
        data = json.loads(tulemus)
        if data and isinstance(data, list):
            return data[0]
        return {}
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print("viga exiftool:", e)
        return {}

def loe_exif_info(droonifail):
    info = {
        'lat': None, 
        'lon': None, 
        'alt': None,
        'fov': None,
        'gimbal_yaw': None, 
        'gimbal_pitch': None, 
        'gimbal_roll': None,
        'flight_yaw': None,
        'flight_pitch': None,
        'flight_roll': None
    }
    meta = exiftool_paring(droonifail)
    if not meta:
        print("EXIF puudub või exiftool error:", droonifail)
        return info

    lat = meta.get('GPSLatitude')
    lon = meta.get('GPSLongitude')
    alt = meta.get('GPSAltitude')
    if lat is not None: info['lat'] = float(lat)
    if lon is not None: info['lon'] = float(lon)
    if alt is not None: info['alt'] = float(alt)

    fv = meta.get('Field Of View')
    if fv is not None:
        try:
            info['fov'] = float(fv)
        except:
            pass

    gy = meta.get('GimbalYawDegree')
    gp = meta.get('GimbalPitchDegree')
    gr = meta.get('GimbalRollDegree')
    fy = meta.get('FlightYawDegree')
    fp = meta.get('FlightPitchDegree')
    fr = meta.get('FlightRollDegree')

    if gy is not None: info['gimbal_yaw'] = float(gy)
    if gp is not None: info['gimbal_pitch'] = float(gp)
    if gr is not None: info['gimbal_roll'] = float(gr)
    if fy is not None: info['flight_yaw'] = float(fy)
    if fp is not None: info['flight_pitch'] = float(fp)
    if fr is not None: info['flight_roll'] = float(fr)

    return info

#2)Koordinaatide teisendus EPSG:4326 -> EPSG:3301 (coordinate transformation)
def proj_3301(lat, lon):
    tr = pyproj.Transformer.from_crs("EPSG:4326","EPSG:3301", always_xy=True)
    x,y = tr.transform(lon, lat)
    return x,y


#3)Maa-ameti WMS hankimine (Maa-amet WMS fetching)
def wms_maaamet_taust(bbox, width=1024, height=1024, kiht="EESTIFOTO"):
    ymin,xmin,ymax,xmax = bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin

    bbox_str= f"{xmin},{ymin},{xmax},{ymax}"

    wms_url= "https://kaart.maaamet.ee/wms/fotokaart"
    params= {
        "service":"WMS",
        "request":"GetMap",
        "version":"1.3.0",
        "layers":kiht,
        "styles":"",
        "crs":"EPSG:3301",
        "bbox": bbox_str,
        "width": str(width),
        "height": str(height),
        "format":"image/png",
        "transparent":"FALSE"
    }
    print("WMS parameetrid:", params)
    resp= requests.get(wms_url, params=params)
    resp.raise_for_status()
    ctype= resp.headers.get("Content-Type","")
    if not ctype.startswith("image/png"):
        print("Ei saa PNG, vaid:", ctype)
        return None
    
    pil_img= Image.open(BytesIO(resp.content)).convert("RGB")
    arr= np.array(pil_img)
    mean_val= arr.mean()
    print(f"WMS pildi keskmine={mean_val:.1f}")
    if mean_val>250:
        print("Hoiatus: pilt valge => ilmselt ei kata ala. Mean>250.")
    return pil_img

#4)Mudeli laadimine = DINOv2_base (Model downloading)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kasutan seadet:", device)

model_name = "facebook/dinov2-base"
print(f"Laen DINOv2 mudelit: {model_name}")
dino_model = AutoModel.from_pretrained(model_name).eval().to(device)

class DinoV2FeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        """
        x => (B,3,H,W)
        esmalt forward output_hidden_states=True,
        siis reshape last_hidden_state => (B, C, h, w).
        """
        outputs = self.backbone(x, output_hidden_states=True)
        hidden = outputs.last_hidden_state   #> (B, seq_len, hidden_dim)
        hidden_no_cls = hidden[:,1:,:]
        B, tokens, dim = hidden_no_cls.shape
        side = int(math.sqrt(tokens))
        feats = hidden_no_cls.reshape(B, side, side, dim)
        feats = feats.permute(0,3,1,2)
        return feats

feat_extractor = DinoV2FeatureExtractor(dino_model).to(device)

map_transform= T.Compose([
    T.Resize((1024,1024)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
drone_transform= T.Compose([
    T.Resize((256,256)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

@torch.no_grad()
def extrakti_omadused(pil_img: Image.Image, transform):
    x = transform(pil_img).unsqueeze(0).to(device)
    feats = feat_extractor(x)
    return feats

#5)Koosinus sarnasus => soojuskaart (CS => heatmap)
def compute_heatmap_ncc(mapF, droneF):
    cross_term = F.conv2d(mapF, droneF, bias=None, stride=1)
    norm_drone = torch.sqrt(torch.sum(droneF**2))

    map_sq = mapF * mapF
    ones_ker = torch.ones_like(droneF)
    sum_map_sq = F.conv2d(map_sq, ones_ker, bias=None, stride=1)
    norm_map = torch.sqrt(sum_map_sq + 1e-8)

    ncc = cross_term / (norm_drone * norm_map + 1e-8)
    heatmap = ncc.squeeze(0).squeeze(0).cpu().numpy()
    return heatmap

#6)arvuta_bbox_fov => coverage ×4
def arvuta_bbox_fov(xLe,yLe, alt_sea, ground_offset=40.0, fov_deg=70.0):
    if alt_sea is None:
        alt_p=200.
    else:
        alt_p= alt_sea-ground_offset
        if alt_p<0:
            alt_p=50.
    rad= math.radians(fov_deg)
    laius= 2.0 * alt_p * math.tan(rad/2.0)
    laius = laius * 4
    pool= laius/2.
    return [xLe-pool, yLe-pool, xLe+pool, yLe+pool]

#7)Droonipildi YAW pööramine (drone image rotate yaw)
def paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=None):
    if yaw is None and flight_yaw is None:
        return droonPil

    if pitch is not None and abs(pitch + 90) < 10:
        angle = 360.0 - yaw if yaw is not None else 0.0
        if flight_yaw is not None:
            yaw_diff = abs((yaw - flight_yaw + 360) % 360)
            if 160 <= yaw_diff <= 200:  
                print(f"Yaw-flip tuvastatud (gimbal: {yaw:.1f}, flight: {flight_yaw:.1f}) => flip +180°")
                angle = (angle + 180) % 360
        print(f"Keeran droonipilti {angle:.1f} kraadi (nadir, gimbal yaw)")
        return droonPil.rotate(angle, expand=True)

    elif flight_yaw is not None:
        angle = 360.0 - flight_yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (flight yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    elif yaw is not None:
        angle = 360.0 - yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (fallback gimbal yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    return droonPil

# 8)pixel2lest => (col,row)->(x,y)
def pixel2lest(bbox, w, h, col, row):
    xmin, ymin, xmax, ymax= bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin
    x= xmin + (xmax - xmin)*(col/ float(w))
    y= ymax - (ymax - ymin)*(row/ float(h))
    return x,y

#9)Drooni teekonna ehitamine (building drone trajectory)
def drooni_teekond_gdf(flight_data):
    geometry=[]
    ajad=[]
    for d in flight_data:
        geometry.append(Point(d['x'], d['y']))
        ajad.append(d['aeg'])
    gdf= gpd.GeoDataFrame({'aeg': ajad}, geometry=geometry, crs="EPSG:3301")
    return gdf

#10)Drooni teekond (drone trajectory)
def joonista_drooni_teekond(gdf, margin=2000):
    minx, miny, maxx, maxy= gdf.total_bounds
    minx-= margin
    miny-= margin
    maxx+= margin
    maxy+= margin

    bbox= [minx, miny, maxx, maxy]
    orto= wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto is None:
        print("Ei saanud WMS => skip.")
        return
    
    arr= np.array(orto)
    fig, ax= plt.subplots(figsize=(10,10))
    ax.imshow(arr, extent=[minx, maxx, miny, maxy], origin='lower')
    gdf.plot(ax=ax, marker='o', color='red', markersize=50, label='Drooni punktid')

    if len(gdf)>1:
        line_geom= LineString(gdf.geometry.values)
        line_gdf= gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:3301")
        line_gdf.plot(ax=ax, color='blue', linewidth=2, label='Drooni teekond')

    for idx, row in gdf.iterrows():
        x_pt= row.geometry.x
        y_pt= row.geometry.y
        ax.text(x_pt, y_pt, str(idx+1), color='white', fontsize=12,
                ha='center', va='center', fontweight='bold')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_title("Drooni teekond (zoomitud, põhjasuunaline)")
    ax.legend()
    plt.show()

#Nihke arvutamine (offset calculation)
offsetid = []
kiirused = []
def arvuta_offset(x_est, y_est, x_gt, y_gt):
    dist = math.sqrt((x_est - x_gt)**2 + (y_est - y_gt)**2)  #Meetrites (in meters)
    return dist

#11)Täielik koodi implementeerimine (full pipeline)
def full_pipeline(kaust, ground_offset=40.0):
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Ei leidu droonifotosid:", kaust)
        return

    flight_data = []

    for fn in failid:
        path = os.path.join(kaust, fn)
        print("\nFail:", path)
        info = loe_exif_info(path)
        lat, lon = info['lat'], info['lon']
        alt = info['alt']
        fov = info['fov'] if info['fov'] else 70
        yaw = info['gimbal_yaw']
        pitch = info['gimbal_pitch']
        roll = info['gimbal_roll']

        if lat is None or lon is None:
            print("Puudub lat/lon => skip.")
            continue

        #1)Katvusala (4x katvus) (bounding box)
        xLe, yLe = proj_3301(lat, lon)
        bbox = arvuta_bbox_fov(xLe, yLe, alt, ground_offset, fov)
        print("=> BBox LEST:", bbox)

        #2)Hangin kaardi (1024×1024, 4x ülekate) (Fetching the map)
        orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
        if orto_pil is None:
            print("Ei saa WMS => skip.")
            continue

        #3)Lae droonipilt, keera ja muuda suurust (256×256) (loading drone image, rotate and resize)
        droonPil = Image.open(path).convert("RGB")
        droonPil = paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=info.get('flight_yaw'))
        droonPil = droonPil.resize((256, 256), Image.BILINEAR)

        #Näita droonipilti ja kaarti kõrvuti (showing the drone picture and map side by side)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
        ax1.imshow(orto_pil)
        ax1.set_title("Kaart => 1024×1024 bounding box (4x ülekate)")
        ax2.imshow(droonPil)
        ax2.set_title("Droon => 256×256")
        plt.show()

        algusaeg = time.time()

        #4)Võtan tunnused välja (shape => (1, C, Hf, Wf)) (Extracting features)
        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droonPil, drone_transform)

        #5)Arvutan heatmapi-slaidiv konvolutsioon (Calculating heatmap-sliding convolution)
        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]
        print(f"Parim simulatsioon => {best_val:.4f}, row={best_i}, col={best_j}")

        loppaeg = time.time()
        kestvus = loppaeg - algusaeg 
        kiirused.append(kestvus)

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='similarity')
        plt.title(f"{fn}: max={best_val:.4f}")
        plt.show()

        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        #Nihke arvutamine tegeliku ja väljund punkti vahel (offset calculating of real and returned red dot)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = arvuta_offset(x_hit, y_hit, x_gt, y_gt)
        offsetid.append(offset)

    if not flight_data:
        print("Pole lennuandmeid -> (exit)")
        return

    gdf = drooni_teekond_gdf(flight_data)
    joonista_drooni_teekond(gdf, margin=2000)

    offsetid_np = np.array(offsetid)
    kiirused_np = np.array(kiirused)

    print("\n==== KOKKUVÕTE ====")
    print(f"Piltide arv: {len(offsetid)}")
    print(f"Keskmine offset: {offsetid_np.mean():.2f} m")
    print(f"Min offset: {offsetid_np.min():.2f} m")
    print(f"Max offset: {offsetid_np.max():.2f} m")
    print(f"Standardhälve: {offsetid_np.std():.2f} m")
    print(f"Keskmine heatmapi genereerimise aeg: {kiirused_np.mean()*1000:.1f} ms")
    print(f"Minimaalne aeg: {kiirused_np.min()*1000:.1f} ms")
    print(f"Maksimaalne aeg: {kiirused_np.max()*1000:.1f} ms")

    #tabelina
    tabel = pd.DataFrame({
        'Fail': failid,
        'Offset_m': offsetid_np,
        'Aeg_ms': kiirused_np * 1000
    })
    print("\nTABEL:")
    print(tabel)

def naide_kasutus():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    full_pipeline(kaust, ground_offset=40)

naide_kasutus()


### DINOv2-small

In [ ]:
# 1)EXIF metaandmete hankimine (EXIF metadata)
def exiftool_paring(failitee):
    try:
        cmd = ["exiftool", "-j", "-n", failitee]
        tulemus = subprocess.check_output(cmd)
        data = json.loads(tulemus)
        if data and isinstance(data, list):
            return data[0]
        return {}
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print("viga exiftool:", e)
        return {}xa

def loe_exif_info(droonifail):
    info = {
        'lat': None, 
        'lon': None, 
        'alt': None,
        'fov': None,
        'gimbal_yaw': None, 
        'gimbal_pitch': None, 
        'gimbal_roll': None,
        'flight_yaw': None,
        'flight_pitch': None,
        'flight_roll': None
    }
    meta = exiftool_paring(droonifail)
    if not meta:
        print("EXIF puudub või exiftool error:", droonifail)
        return info

    lat = meta.get('GPSLatitude')
    lon = meta.get('GPSLongitude')
    alt = meta.get('GPSAltitude')
    if lat is not None: info['lat'] = float(lat)
    if lon is not None: info['lon'] = float(lon)
    if alt is not None: info['alt'] = float(alt)

    fv = meta.get('Field Of View')
    if fv is not None:
        try:
            info['fov'] = float(fv)
        except:
            pass

    gy = meta.get('GimbalYawDegree')
    gp = meta.get('GimbalPitchDegree')
    gr = meta.get('GimbalRollDegree')
    fy = meta.get('FlightYawDegree')
    fp = meta.get('FlightPitchDegree')
    fr = meta.get('FlightRollDegree')

    if gy is not None: info['gimbal_yaw'] = float(gy)
    if gp is not None: info['gimbal_pitch'] = float(gp)
    if gr is not None: info['gimbal_roll'] = float(gr)
    if fy is not None: info['flight_yaw'] = float(fy)
    if fp is not None: info['flight_pitch'] = float(fp)
    if fr is not None: info['flight_roll'] = float(fr)

    return info

#2)Koordinaatide teisendus EPSG:4326 -> EPSG:3301 (coordinate transformation)
def proj_3301(lat, lon):
    tr = pyproj.Transformer.from_crs("EPSG:4326","EPSG:3301", always_xy=True)
    x,y = tr.transform(lon, lat)
    return x,y


#3)Maa-ameti WMS hankimine (Maa-amet WMS fetching)
def wms_maaamet_taust(bbox, width=1024, height=1024, kiht="EESTIFOTO"):
    ymin,xmin,ymax,xmax = bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin

    bbox_str= f"{xmin},{ymin},{xmax},{ymax}"

    wms_url= "https://kaart.maaamet.ee/wms/fotokaart"
    params= {
        "service":"WMS",
        "request":"GetMap",
        "version":"1.3.0",
        "layers":kiht,
        "styles":"",
        "crs":"EPSG:3301",
        "bbox": bbox_str,
        "width": str(width),
        "height": str(height),
        "format":"image/png",
        "transparent":"FALSE"
    }
    print("WMS parameetrid:", params)
    resp= requests.get(wms_url, params=params)
    resp.raise_for_status()
    ctype= resp.headers.get("Content-Type","")
    if not ctype.startswith("image/png"):
        print("Ei saa PNG, vaid:", ctype)
        return None
    
    pil_img= Image.open(BytesIO(resp.content)).convert("RGB")
    arr= np.array(pil_img)
    mean_val= arr.mean()
    print(f"WMS pildi keskmine={mean_val:.1f}")
    if mean_val>250:
        print("Hoiatus: pilt valge => ilmselt ei kata ala. Mean>250.")
    return pil_img

#4)Mudeli laadimine = DINOv2-väike (Model downloading)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kasutan seadet:", device)

model_name = "facebook/dinov2-small"  
print(f"Laen DINOv2 mudelit: {model_name}")
dino_model = AutoModel.from_pretrained(model_name).eval().to(device)

class DinoV2FeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        """
        x => (B,3,H,W)
        esmalt forward output_hidden_states=True,
        siis reshape last_hidden_state => (B, C, h, w).
        """
        outputs = self.backbone(x, output_hidden_states=True)
        hidden = outputs.last_hidden_state   #> (B, seq_len, hidden_dim)
        hidden_no_cls = hidden[:,1:,:]
        B, tokens, dim = hidden_no_cls.shape
        side = int(math.sqrt(tokens))
        feats = hidden_no_cls.reshape(B, side, side, dim)
        feats = feats.permute(0,3,1,2)
        return feats

feat_extractor = DinoV2FeatureExtractor(dino_model).to(device)

map_transform= T.Compose([
    T.Resize((1024,1024)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
drone_transform= T.Compose([
    T.Resize((256,256)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

@torch.no_grad()
def extrakti_omadused(pil_img: Image.Image, transform):
    x = transform(pil_img).unsqueeze(0).to(device)
    feats = feat_extractor(x)
    return feats

#5)koosinus sarnasus => soojuskaart (cosine similarity to heatmap)
def compute_heatmap_ncc(mapF, droneF):
    cross_term = F.conv2d(mapF, droneF, bias=None, stride=1)
    norm_drone = torch.sqrt(torch.sum(droneF**2))

    map_sq = mapF * mapF
    ones_ker = torch.ones_like(droneF)
    sum_map_sq = F.conv2d(map_sq, ones_ker, bias=None, stride=1)
    norm_map = torch.sqrt(sum_map_sq + 1e-8)

    ncc = cross_term / (norm_drone * norm_map + 1e-8)
    heatmap = ncc.squeeze(0).squeeze(0).cpu().numpy()
    return heatmap

#6)arvuta_bbox_fov => katvus ×4
def arvuta_bbox_fov(xLe,yLe, alt_sea, ground_offset=40.0, fov_deg=70.0):
    if alt_sea is None:
        alt_p=200.
    else:
        alt_p= alt_sea-ground_offset
        if alt_p<0:
            alt_p=50.
    rad= math.radians(fov_deg)
    laius= 2.0 * alt_p * math.tan(rad/2.0)
    laius = laius * 4
    pool= laius/2.
    return [xLe-pool, yLe-pool, xLe+pool, yLe+pool]

#7)Droonipildi YAW pööramine (drone image rotate yaw)
def paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=None):
    if yaw is None and flight_yaw is None:
        return droonPil

    #Kontroll: kas on vaatamas otse alla 
    if pitch is not None and abs(pitch + 90) < 10:
        angle = 360.0 - yaw if yaw is not None else 0.0
        if flight_yaw is not None:
            yaw_diff = abs((yaw - flight_yaw + 360) % 360)
            if 160 <= yaw_diff <= 200:  
                print(f"Yaw-flip tuvastatud (gimbal: {yaw:.1f}, flight: {flight_yaw:.1f}) => flip +180°")
                angle = (angle + 180) % 360
        print(f"Keeran droonipilti {angle:.1f} kraadi (nadir, gimbal yaw)")
        return droonPil.rotate(angle, expand=True)

    elif flight_yaw is not None:
        angle = 360.0 - flight_yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (flight yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    elif yaw is not None:
        angle = 360.0 - yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (fallback gimbal yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    return droonPil

#8)pixel2lest => (col,row)->(x,y)
def pixel2lest(bbox, w, h, col, row):
    xmin, ymin, xmax, ymax= bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin
    x= xmin + (xmax - xmin)*(col/ float(w))
    y= ymax - (ymax - ymin)*(row/ float(h))
    return x,y

#9)Ehitan drooni teekonda (Building drone trajectory)
def drooni_teekond_gdf(flight_data):
    geometry=[]
    ajad=[]
    for d in flight_data:
        geometry.append(Point(d['x'], d['y']))
        ajad.append(d['aeg'])
    gdf= gpd.GeoDataFrame({'aeg': ajad}, geometry=geometry, crs="EPSG:3301")
    return gdf

#10)Drooni teekond (Drone trajectory)
def joonista_drooni_teekond(gdf, margin=2000):
    minx, miny, maxx, maxy= gdf.total_bounds
    minx-= margin
    miny-= margin
    maxx+= margin
    maxy+= margin

    bbox= [minx, miny, maxx, maxy]
    orto= wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto is None:
        print("Ei saanud WMS => skip.")
        return
    
    arr= np.array(orto)
    fig, ax= plt.subplots(figsize=(10,10))
    ax.imshow(arr, extent=[minx, maxx, miny, maxy], origin='lower')
    gdf.plot(ax=ax, marker='o', color='red', markersize=50, label='Drooni punktid')

    if len(gdf)>1:
        line_geom= LineString(gdf.geometry.values)
        line_gdf= gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:3301")
        line_gdf.plot(ax=ax, color='blue', linewidth=2, label='Drooni teekond')

    for idx, row in gdf.iterrows():
        x_pt= row.geometry.x
        y_pt= row.geometry.y
        ax.text(x_pt, y_pt, str(idx+1), color='white', fontsize=12,
                ha='center', va='center', fontweight='bold')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_title("Drooni teekond (zoomitud, põhjasuunaline)")
    ax.legend()
    plt.show()

#Nihke arvutamine (Offset calulation)
offsetid = []
kiirused = []
def arvuta_offset(x_est, y_est, x_gt, y_gt):
    dist = math.sqrt((x_est - x_gt)**2 + (y_est - y_gt)**2)  
    return dist

#11)Täielik koodi implementeerimine (Full pipeline)
def full_pipeline(kaust, ground_offset=40.0):
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Ei leidu droonifotosid:", kaust)
        return

    flight_data = []

    for fn in failid:
        path = os.path.join(kaust, fn)
        print("\nFail:", path)
        info = loe_exif_info(path)
        lat, lon = info['lat'], info['lon']
        alt = info['alt']
        fov = info['fov'] if info['fov'] else 70
        yaw = info['gimbal_yaw']
        pitch = info['gimbal_pitch']
        roll = info['gimbal_roll']

        if lat is None or lon is None:
            print("Puudub lat/lon => skip.")
            continue

        xLe, yLe = proj_3301(lat, lon)
        bbox = arvuta_bbox_fov(xLe, yLe, alt, ground_offset, fov)
        print("=> BBox LEST:", bbox)

        orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
        if orto_pil is None:
            print("Ei saa WMS => skip.")
            continue

        droonPil = Image.open(path).convert("RGB")
        droonPil = paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=info.get('flight_yaw'))
        droonPil = droonPil.resize((256, 256), Image.BILINEAR)

        #Droonipilt ja kaart kõrvuti (drone image and map besides each other)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
        ax1.imshow(orto_pil)
        ax1.set_title("Kaart => 1024×1024 bounding box (4x ülekate)")
        ax2.imshow(droonPil)
        ax2.set_title("Droon => 256×256")
        plt.show()

        algusaeg = time.time()

        #4)Tunnuste eraldamine (shape => (1, C, Hf, Wf)) (Feature extraction)
        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droonPil, drone_transform)

        #5)Arvutan soojuskaardi (calculating a heatmpat) 
        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]
        print(f"Parim simulatsioon => {best_val:.4f}, row={best_i}, col={best_j}")

        loppaeg = time.time()
        kestvus = loppaeg - algusaeg  
        kiirused.append(kestvus)

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='similarity')
        plt.title(f"{fn}: max={best_val:.4f}")
        plt.show()

        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        #Nihke arvutmaine tegeliku ja genereeritud punkti vahel (offset calculation)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = arvuta_offset(x_hit, y_hit, x_gt, y_gt)
        offsetid.append(offset)

    if not flight_data:
        print("Pole lennuandmeid -> (exit)")
        return

    gdf = drooni_teekond_gdf(flight_data)
    joonista_drooni_teekond(gdf, margin=2000)

    offsetid_np = np.array(offsetid)
    kiirused_np = np.array(kiirused)

    print("\n==== KOKKUVÕTE ====")
    print(f"Piltide arv: {len(offsetid)}")
    print(f"Keskmine offset: {offsetid_np.mean():.2f} m")
    print(f"Min offset: {offsetid_np.min():.2f} m")
    print(f"Max offset: {offsetid_np.max():.2f} m")
    print(f"Standardhälve: {offsetid_np.std():.2f} m")
    print(f"Keskmine heatmapi genereerimise aeg: {kiirused_np.mean()*1000:.1f} ms")
    print(f"Minimaalne aeg: {kiirused_np.min()*1000:.1f} ms")
    print(f"Maksimaalne aeg: {kiirused_np.max()*1000:.1f} ms")

    tabel = pd.DataFrame({
        'Fail': failid,
        'Offset_m': offsetid_np,
        'Aeg_ms': kiirused_np * 1000
    })
    print("\nTABEL:")
    print(tabel)

def naide_kasutus():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    full_pipeline(kaust, ground_offset=40)

naide_kasutus()


### DINOv2-large

In [ ]:
# 1)EXIF metaandmed (EXIF metadata)
def exiftool_paring(failitee):
    try:
        cmd = ["exiftool", "-j", "-n", failitee]
        tulemus = subprocess.check_output(cmd)
        data = json.loads(tulemus)
        if data and isinstance(data, list):
            return data[0]
        return {}
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print("viga exiftool:", e)
        return {}

def loe_exif_info(droonifail):
    info = {
        'lat': None, 
        'lon': None, 
        'alt': None,
        'fov': None,
        'gimbal_yaw': None, 
        'gimbal_pitch': None, 
        'gimbal_roll': None,
        'flight_yaw': None,
        'flight_pitch': None,
        'flight_roll': None
    }
    meta = exiftool_paring(droonifail)
    if not meta:
        print("EXIF puudub või exiftool error:", droonifail)
        return info

    lat = meta.get('GPSLatitude')
    lon = meta.get('GPSLongitude')
    alt = meta.get('GPSAltitude')
    if lat is not None: info['lat'] = float(lat)
    if lon is not None: info['lon'] = float(lon)
    if alt is not None: info['alt'] = float(alt)

    fv = meta.get('Field Of View')
    if fv is not None:
        try:
            info['fov'] = float(fv)
        except:
            pass

    gy = meta.get('GimbalYawDegree')
    gp = meta.get('GimbalPitchDegree')
    gr = meta.get('GimbalRollDegree')
    fy = meta.get('FlightYawDegree')
    fp = meta.get('FlightPitchDegree')
    fr = meta.get('FlightRollDegree')

    if gy is not None: info['gimbal_yaw'] = float(gy)
    if gp is not None: info['gimbal_pitch'] = float(gp)
    if gr is not None: info['gimbal_roll'] = float(gr)
    if fy is not None: info['flight_yaw'] = float(fy)
    if fp is not None: info['flight_pitch'] = float(fp)
    if fr is not None: info['flight_roll'] = float(fr)

    return info

#2)Koordinaatide teisendus EPSG:4326 -> EPSG:3301 (coordinate transformation)
def proj_3301(lat, lon):
    tr = pyproj.Transformer.from_crs("EPSG:4326","EPSG:3301", always_xy=True)
    x,y = tr.transform(lon, lat)
    return x,y


#3)Maa-ameti WMS hankimine (Maa-amet WMS fetching)
def wms_maaamet_taust(bbox, width=1024, height=1024, kiht="EESTIFOTO"):
    ymin,xmin,ymax,xmax = bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin

    bbox_str= f"{xmin},{ymin},{xmax},{ymax}"

    wms_url= "https://kaart.maaamet.ee/wms/fotokaart"
    params= {
        "service":"WMS",
        "request":"GetMap",
        "version":"1.3.0",
        "layers":kiht,
        "styles":"",
        "crs":"EPSG:3301",
        "bbox": bbox_str,
        "width": str(width),
        "height": str(height),
        "format":"image/png",
        "transparent":"FALSE"
    }
    print("WMS parameetrid:", params)
    resp= requests.get(wms_url, params=params)
    resp.raise_for_status()
    ctype= resp.headers.get("Content-Type","")
    if not ctype.startswith("image/png"):
        print("Ei saa PNG, vaid:", ctype)
        return None
    
    pil_img= Image.open(BytesIO(resp.content)).convert("RGB")
    arr= np.array(pil_img)
    mean_val= arr.mean()
    print(f"WMS pildi keskmine={mean_val:.1f}")
    if mean_val>250:
        print("Hoiatus: pilt valge => ilmselt ei kata ala. Mean>250.")
    return pil_img

#4)Mudeli laadimine = DINOv2-suur (Model downloading)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kasutan seadet:", device)

model_name = "facebook/dinov2-large"  
print(f"Laen DINOv2 mudelit: {model_name}")
dino_model = AutoModel.from_pretrained(model_name).eval().to(device)

class DinoV2FeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        """
        x => (B,3,H,W)
        esmalt forward output_hidden_states=True,
        siis reshape last_hidden_state => (B, C, h, w).
        """
        outputs = self.backbone(x, output_hidden_states=True)
        hidden = outputs.last_hidden_state   #> (B, seq_len, hidden_dim)
        hidden_no_cls = hidden[:,1:,:]
        B, tokens, dim = hidden_no_cls.shape
        side = int(math.sqrt(tokens))
        feats = hidden_no_cls.reshape(B, side, side, dim)
        feats = feats.permute(0,3,1,2)
        return feats

feat_extractor = DinoV2FeatureExtractor(dino_model).to(device)


map_transform= T.Compose([
    T.Resize((1024,1024)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
drone_transform= T.Compose([
    T.Resize((256,256)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

@torch.no_grad()
def extrakti_omadused(pil_img: Image.Image, transform):
    x = transform(pil_img).unsqueeze(0).to(device)
    feats = feat_extractor(x)
    return feats

#5)koosinus sarnasus => soojuskaart (CS to heatmap)
def compute_heatmap_ncc(mapF, droneF):
    cross_term = F.conv2d(mapF, droneF, bias=None, stride=1)
    norm_drone = torch.sqrt(torch.sum(droneF**2))

    map_sq = mapF * mapF
    ones_ker = torch.ones_like(droneF)
    sum_map_sq = F.conv2d(map_sq, ones_ker, bias=None, stride=1)
    norm_map = torch.sqrt(sum_map_sq + 1e-8)

    ncc = cross_term / (norm_drone * norm_map + 1e-8)
    heatmap = ncc.squeeze(0).squeeze(0).cpu().numpy()
    return heatmap

# 6)arvuta_bbox_fov => coverage ×4
def arvuta_bbox_fov(xLe,yLe, alt_sea, ground_offset=40.0, fov_deg=70.0):
    if alt_sea is None:
        alt_p=200.
    else:
        alt_p= alt_sea-ground_offset
        if alt_p<0:
            alt_p=50.
    rad= math.radians(fov_deg)
    laius= 2.0 * alt_p * math.tan(rad/2.0)
    laius = laius * 4
    pool= laius/2.
    return [xLe-pool, yLe-pool, xLe+pool, yLe+pool]

# 7)Droonipildi YAW pööramine (Drone image YAW fixing)
def paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=None):
    if yaw is None and flight_yaw is None:
        return droonPil

    if pitch is not None and abs(pitch + 90) < 10:
        angle = 360.0 - yaw if yaw is not None else 0.0
        if flight_yaw is not None:
            yaw_diff = abs((yaw - flight_yaw + 360) % 360)
            if 160 <= yaw_diff <= 200:  
                print(f"Yaw-flip tuvastatud (gimbal: {yaw:.1f}, flight: {flight_yaw:.1f}) => flip +180°")
                angle = (angle + 180) % 360
        print(f"Keeran droonipilti {angle:.1f} kraadi (nadir, gimbal yaw)")
        return droonPil.rotate(angle, expand=True)

    elif flight_yaw is not None:
        angle = 360.0 - flight_yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (flight yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    elif yaw is not None:
        angle = 360.0 - yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (fallback gimbal yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)

    return droonPil

# 8)pixel2lest => (col,row)->(x,y)
def pixel2lest(bbox, w, h, col, row):
    xmin, ymin, xmax, ymax= bbox
    if xmin> xmax:
        xmin,xmax= xmax,xmin
    if ymin> ymax:
        ymin,ymax= ymax,ymin
    x= xmin + (xmax - xmin)*(col/ float(w))
    y= ymax - (ymax - ymin)*(row/ float(h))
    return x,y

#9)Ehitan drooni teekonda (Building drone trajectory)
def drooni_teekond_gdf(flight_data):
    geometry=[]
    ajad=[]
    for d in flight_data:
        geometry.append(Point(d['x'], d['y']))
        ajad.append(d['aeg'])
    gdf= gpd.GeoDataFrame({'aeg': ajad}, geometry=geometry, crs="EPSG:3301")
    return gdf

#10)Drooni teekond (Drone trajectory)
def joonista_drooni_teekond(gdf, margin=2000):
    minx, miny, maxx, maxy= gdf.total_bounds
    minx-= margin
    miny-= margin
    maxx+= margin
    maxy+= margin

    bbox= [minx, miny, maxx, maxy]
    orto= wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto is None:
        print("Ei saanud WMS => skip.")
        return
    
    arr= np.array(orto)
    fig, ax= plt.subplots(figsize=(10,10))
    ax.imshow(arr, extent=[minx, maxx, miny, maxy], origin='lower')
    gdf.plot(ax=ax, marker='o', color='red', markersize=50, label='Drooni punktid')

    if len(gdf)>1:
        line_geom= LineString(gdf.geometry.values)
        line_gdf= gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:3301")
        line_gdf.plot(ax=ax, color='blue', linewidth=2, label='Drooni teekond')

    for idx, row in gdf.iterrows():
        x_pt= row.geometry.x
        y_pt= row.geometry.y
        ax.text(x_pt, y_pt, str(idx+1), color='white', fontsize=12,
                ha='center', va='center', fontweight='bold')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_title("Drooni teekond (zoomitud, põhjasuunaline)")
    ax.legend()
    plt.show()

#Nihke arvutamine (Offset calculation)
offsetid = []
kiirused = []
def arvuta_offset(x_est, y_est, x_gt, y_gt):
    dist = math.sqrt((x_est - x_gt)**2 + (y_est - y_gt)**2)  
    return dist

#11)Täielik koodi implementeerimine (Full pipeline)
def full_pipeline(kaust, ground_offset=40.0):
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Ei leidu droonifotosid:", kaust)
        return

    flight_data = []

    for fn in failid:
        path = os.path.join(kaust, fn)
        print("\nFail:", path)
        info = loe_exif_info(path)
        lat, lon = info['lat'], info['lon']
        alt = info['alt']
        fov = info['fov'] if info['fov'] else 70
        yaw = info['gimbal_yaw']
        pitch = info['gimbal_pitch']
        roll = info['gimbal_roll']

        if lat is None or lon is None:
            print("Puudub lat/lon => skip.")
            continue

        xLe, yLe = proj_3301(lat, lon)
        bbox = arvuta_bbox_fov(xLe, yLe, alt, ground_offset, fov)
        print("=> BBox LEST:", bbox)

        orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
        if orto_pil is None:
            print("Ei saa WMS => skip.")
            continue

        droonPil = Image.open(path).convert("RGB")
        droonPil = paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=info.get('flight_yaw'))
        droonPil = droonPil.resize((256, 256), Image.BILINEAR)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
        ax1.imshow(orto_pil)
        ax1.set_title("Kaart => 1024×1024 bounding box (4x ülekate)")
        ax2.imshow(droonPil)
        ax2.set_title("Droon => 256×256")
        plt.show()

        algusaeg = time.time()

        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droonPil, drone_transform)

        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]
        print(f"Parim simulatsioon => {best_val:.4f}, row={best_i}, col={best_j}")

        loppaeg = time.time()
        kestvus = loppaeg - algusaeg  
        kiirused.append(kestvus)

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='similarity')
        plt.title(f"{fn}: max={best_val:.4f}")
        plt.show()

        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        x_gt, y_gt = proj_3301(lat, lon)
        offset = arvuta_offset(x_hit, y_hit, x_gt, y_gt)
        offsetid.append(offset)

    if not flight_data:
        print("Pole lennuandmeid -> (exit)")
        return

    gdf = drooni_teekond_gdf(flight_data)
    joonista_drooni_teekond(gdf, margin=2000)

    offsetid_np = np.array(offsetid)
    kiirused_np = np.array(kiirused)

    print("\n==== KOKKUVÕTE ====")
    print(f"Piltide arv: {len(offsetid)}")
    print(f"Keskmine offset: {offsetid_np.mean():.2f} m")
    print(f"Min offset: {offsetid_np.min():.2f} m")
    print(f"Max offset: {offsetid_np.max():.2f} m")
    print(f"Standardhälve: {offsetid_np.std():.2f} m")
    print(f"Keskmine heatmapi genereerimise aeg: {kiirused_np.mean()*1000:.1f} ms")
    print(f"Minimaalne aeg: {kiirused_np.min()*1000:.1f} ms")
    print(f"Maksimaalne aeg: {kiirused_np.max()*1000:.1f} ms")

    tabel = pd.DataFrame({
        'Fail': failid,
        'Offset_m': offsetid_np,
        'Aeg_ms': kiirused_np * 1000
    })
    print("\nTABEL:")
    print(tabel)

def naide_kasutus():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    full_pipeline(kaust, ground_offset=40)

naide_kasutus()


#### Eksperiment: Drooni erinevate lennukõrguste mõju tulemustele

In [ ]:
offsetid_per_height = {}

def testi_erinevad_korgused(droonifail, korgused=[120, 80, 60, 40, 30, 20, 10]):
    global offsetid_per_height
    info = loe_exif_info(droonifail)
    lat, lon = info['lat'], info['lon']
    yaw = info['gimbal_yaw'] or 0.0
    pitch = info['gimbal_pitch'] or -90.0
    roll = info['gimbal_roll'] or 0.0
    fov = info['fov'] or 70.0

    if lat is None or lon is None:
        print("Puudub GPS info pildil")
        return

    droon_orig_full = Image.open(droonifail).convert("RGB")
    droon_orig_full = paranda_yaw_kui_nadir(droon_orig_full, yaw, pitch, roll)

    xLe, yLe = proj_3301(lat, lon)

    bbox = arvuta_bbox_fov(xLe, yLe, korgused[0], ground_offset=0, fov_deg=fov)
    orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto_pil is None:
        print("Ei saanud WMS-kaarti")
        return

    for alt in korgused:
        print(f"\nTestin kõrgusel: {alt} m")

        rad = math.radians(fov / 2.0)
        vaade_m = 2.0 * alt * math.tan(rad) 
        px_per_m = droon_orig_full.width / (2.0 * info['alt'] * math.tan(rad))
        crop_px = int(vaade_m * px_per_m)

        kesk_x = droon_orig_full.width // 2
        kesk_y = droon_orig_full.height // 2
        pool = crop_px // 2
        crop = droon_orig_full.crop((kesk_x - pool, kesk_y - pool, kesk_x + pool, kesk_y + pool))
        crop = crop.resize((256, 256), Image.BILINEAR)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(orto_pil)
        ax1.set_title(f"Kaart (fikseeritud bbox, alt={korgused[0]}m)")
        ax2.imshow(crop)
        ax2.set_title(f"Droonipilt (zoomitud, alt={alt}m)")
        plt.show()

        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(crop, drone_transform)

        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]

        print(f"Parim sarnasus alt {alt} m => {best_val:.4f}, pixel pos: {best_j},{best_i}")

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='similarity')
        plt.title(f"Heatmap (alt {alt} m): max={best_val:.4f}")
        plt.show()

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = math.hypot(x_hit - x_gt, y_hit - y_gt)
        print(f"Offset kõrgusel {alt}m: {offset:.2f} m")

        if alt not in offsetid_per_height:
            offsetid_per_height[alt] = []
        offsetid_per_height[alt].append(offset)

def naide_korgus_test():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Pole faile kaustas.")
        return

    failid = failid[:10]
    print(f"Testin esimesed {len(failid)} faili\n")

    for fail in failid:
        droonifail = os.path.join(kaust, fail)
        print(f"\nTestin: {droonifail}")
        testi_erinevad_korgused(droonifail)

    print("\n=== TULEMUSED KÕRGUSTE KAUPA ===")
    summary_rows = []
    all_offsets = []
    for alt in sorted(offsetid_per_height.keys(), reverse=True):
        offsets = np.array(offsetid_per_height[alt])
        all_offsets.extend(offsets)
        print(f"\nKõrgus {alt} m:")
        print(f"  Arv: {len(offsets)}")
        print(f"  Keskmine offset: {offsets.mean():.2f} m")
        print(f"  Mediaan offset: {np.median(offsets):.2f} m")
        print(f"  Min offset: {offsets.min():.2f} m")
        print(f"  Max offset: {offsets.max():.2f} m")
        print(f"  Standardhälve: {offsets.std():.2f} m")
        summary_rows.append({
            "Kõrgus (m)": alt,
            "Keskmine (m)": offsets.mean(),
            "Mediaan (m)": np.median(offsets),
            "Min (m)": offsets.min(),
            "Max (m)": offsets.max(),
            "Std (m)": offsets.std(),
            "Kogus": len(offsets)
        })

    tabel = pd.DataFrame(summary_rows)
    print("\nKokkuvõtte tabel (per kõrgus):\n", tabel)

    if len(all_offsets) > 0:
        all_offsets = np.array(all_offsets)
        overall_summary = pd.DataFrame([{
            "Kõrgus (m)": "Kõik kõrgused kokku",
            "Keskmine (m)": all_offsets.mean(),
            "Mediaan (m)": np.median(all_offsets),
            "Min (m)": all_offsets.min(),
            "Max (m)": all_offsets.max(),
            "Std (m)": all_offsets.std(),
            "Kogus": len(all_offsets)
        }])
        print("\n=== ÜLDTABEL KÕIKIDE KÕRGUSTE KOHTA ===")
        print(overall_summary)
    else:
        print("\nPole ühtegi offseti, üldtabelit ei saa luua.")

naide_korgus_test()


#### Eksperiment: Drooni eri pöördenurkade mõju tulemustele

In [ ]:
offsetid_per_yaw = {}

def testi_erinevad_yawid(droonifail, test_yawid=[1, 2, 3, 4, 5, 10, 20, 30, 40]):
    global offsetid_per_yaw
    info = loe_exif_info(droonifail)
    lat, lon = info['lat'], info['lon']
    orig_alt = info['alt'] or 120.0
    pitch = info['gimbal_pitch'] or -90.0
    roll = info['gimbal_roll'] or 0.0
    fov = info['fov'] or 70.0

    if lat is None or lon is None:
        print("Puudub GPS info pildil")
        return

    droon_orig_full = Image.open(droonifail).convert("RGB")

    xLe, yLe = proj_3301(lat, lon)
    bbox = arvuta_bbox_fov(xLe, yLe, orig_alt, ground_offset=0, fov_deg=fov)
    orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
    if orto_pil is None:
        print("Ei saanud WMS-kaarti")
        return

    rad = math.radians(fov / 2.0)
    vaade_m = 2.0 * orig_alt * math.tan(rad)
    px_per_m = droon_orig_full.width / (2.0 * orig_alt * math.tan(rad))
    crop_px = int(vaade_m * px_per_m)

    kesk_x = droon_orig_full.width // 2
    kesk_y = droon_orig_full.height // 2
    pool = crop_px // 2
    crop_orig = droon_orig_full.crop((kesk_x - pool, kesk_y - pool, kesk_x + pool, kesk_y + pool))

    for yaw_test in test_yawid:
        print(f"\nTestin yaw: {yaw_test} kraadi")

        droon_crop = paranda_yaw_kui_nadir(crop_orig, yaw=yaw_test, pitch=pitch, roll=roll)
        droon_crop = droon_crop.resize((256, 256), Image.BILINEAR)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(orto_pil)
        ax1.set_title(f"Kaart (alt={orig_alt}m)")
        ax2.imshow(droon_crop)
        ax2.set_title(f"Droonipilt (yaw={yaw_test}°)")
        plt.show()

        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droon_crop, drone_transform)

        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]

        print(f"Parim sarnasus yaw={yaw_test}° => {best_val:.4f}")

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='similarity')
        plt.title(f"Heatmap (yaw={yaw_test}°): max={best_val:.4f}")
        plt.show()

        w = simmap.shape[1]
        h = simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = math.hypot(x_hit - x_gt, y_hit - y_gt)
        print(f"Offset yaw {yaw_test}°: {offset:.2f} m")

        if yaw_test not in offsetid_per_yaw:
            offsetid_per_yaw[yaw_test] = []
        offsetid_per_yaw[yaw_test].append(offset)


def naide_yaw_test():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Pole faile kaustas.")
        return

    failid = failid[:10]
    print(f"Testin esimesed {len(failid)} faili\n")

    for fail in failid:
        droonifail = os.path.join(kaust, fail)
        print(f"\nTestin: {droonifail}")
        testi_erinevad_yawid(droonifail)

    print("\n=== TULEMUSED YAWIDE KAUPA ===")
    summary_rows = []
    all_offsets = []
    for yaw in sorted(offsetid_per_yaw.keys()):
        offsets = np.array(offsetid_per_yaw[yaw])
        all_offsets.extend(offsets)
        print(f"\nYaw {yaw}°:")
        print(f"  Arv: {len(offsets)}")
        print(f"  Keskmine offset: {offsets.mean():.2f} m")
        print(f"  Mediaan offset: {np.median(offsets):.2f} m")
        print(f"  Min offset: {offsets.min():.2f} m")
        print(f"  Max offset: {offsets.max():.2f} m")
        print(f"  Standardhälve: {offsets.std():.2f} m")
        summary_rows.append({
            "Yaw (°)": yaw,
            "Keskmine (m)": offsets.mean(),
            "Mediaan (m)": np.median(offsets),
            "Min (m)": offsets.min(),
            "Max (m)": offsets.max(),
            "Std (m)": offsets.std(),
            "Kogus": len(offsets)
        })

    tabel = pd.DataFrame(summary_rows)
    print("\nKokkuvõtte tabel (per yaw):\n", tabel)

    if len(all_offsets) > 0:
        all_offsets = np.array(all_offsets)
        overall_summary = pd.DataFrame([{
            "Yaw (°)": "Kõik yaw väärtused kokku",
            "Keskmine (m)": all_offsets.mean(),
            "Mediaan (m)": np.median(all_offsets),
            "Min (m)": all_offsets.min(),
            "Max (m)": all_offsets.max(),
            "Std (m)": all_offsets.std(),
            "Kogus": len(all_offsets)
        }])
        print("\n=== ÜLDTABEL KÕIKIDE YAWIDE KOHTA ===")
        print(overall_summary)
    else:
        print("\nPole ühtegi offseti, üldtabelit ei saa luua.")

naide_yaw_test()


## WALDO30 (YOLOv8)

In [ ]:
#1)EXIF metaandmete lugemine (exif metadata reading)
def exiftool_paring(failitee):
    try:
        cmd = ["exiftool", "-j", "-n", failitee]
        tulemus = subprocess.check_output(cmd)
        data = json.loads(tulemus)
        if data and isinstance(data, list):
            return data[0]
        return {}
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print("Viga exiftool:", e)
        return {}

def loe_exif_info(droonifail):
    info = {'lat': None, 'lon': None, 'alt': None,
            'fov': None, 'gimbal_yaw': None, 'gimbal_pitch': None, 'gimbal_roll': None}
    meta = exiftool_paring(droonifail)
    if not meta:
        print("EXIF puudub või exiftool error:", droonifail)
        return info
    lat = meta.get('GPSLatitude')
    lon = meta.get('GPSLongitude')
    alt = meta.get('GPSAltitude')
    if lat is not None: info['lat'] = float(lat)
    if lon is not None: info['lon'] = float(lon)
    if alt is not None: info['alt'] = float(alt)
    fv = meta.get('Field Of View')
    if fv is not None:
        try: info['fov'] = float(fv)
        except: pass
    gy = meta.get('GimbalYawDegree')
    gp = meta.get('GimbalPitchDegree')
    gr = meta.get('GimbalRollDegree')
    if gy is not None: info['gimbal_yaw'] = float(gy)
    if gp is not None: info['gimbal_pitch'] = float(gp)
    if gr is not None: info['gimbal_roll'] = float(gr)
    return info

#2)Koordinaatide teisendus EPSG:4326 -> EPSG:3301 (coordinate transformation)
def proj_3301(lat, lon):
    tr = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3301", always_xy=True)
    return tr.transform(lon, lat)

#3)WMS pildi küsimine Maaametist (WMS photo fetching)
def wms_maaamet_taust(bbox, width=1024, height=1024, kiht="EESTIFOTO"):
    ymin, xmin, ymax, xmax = bbox  
    if xmin > xmax: xmin, xmax = xmax, xmin
    if ymin > ymax: ymin, ymax = ymax, ymin
    bbox_str = f"{xmin},{ymin},{xmax},{ymax}"

    params = {
        "service": "WMS", "request": "GetMap", "version": "1.3.0",
        "layers": kiht, "styles": "", "crs": "EPSG:3301",
        "bbox": bbox_str, "width": str(width), "height": str(height),
        "format": "image/png", "transparent": "FALSE"
    }
    resp = requests.get("https://kaart.maaamet.ee/wms/fotokaart", params=params)
    resp.raise_for_status()
    if not resp.headers.get("Content-Type", "").startswith("image/png"):
        print("Ei saanud PNG:", resp.headers.get("Content-Type"))
        return None
    pil_img = Image.open(BytesIO(resp.content)).convert("RGB")
    arr = np.array(pil_img)
    print(f"WMS pildi keskmine heledus={arr.mean():.1f}")
    if arr.mean() > 250:
        print("Hoiatus: pilt liiga hele (võib olla tühi bbox)")
    return pil_img

#4)YOLO mudeli laadimine (YOLO model)
seade = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kasutan seadet:", seade)

repo_id = "StephanST/WALDO30"
filename = "WALDO30_yolov8m_640x640.pt"
my_token = "LISA KASUTAMISEKS ENDA TOKEN"
local_pt_path = hf_hub_download(repo_id=repo_id, filename=filename, token=my_token)
print("Laetud YOLO mudel:", local_pt_path)

model_yolo = YOLO(local_pt_path).to(seade).eval()

#4.1) Ühe kihi feature hook (one layer feature hook)
class SingleFeatureHook:
    def __init__(self, module):
        self.module = module
        self.features = None
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()

layer_index = 12
chosen_layer = model_yolo.model.model[layer_index]
print(f"Valitud YOLO kiht indeksiga {layer_index}: {chosen_layer}")
feature_hook = SingleFeatureHook(chosen_layer)

class Waldo30SingleLayerExtractor(nn.Module):
    def __init__(self, yolo_model, hook):
        super().__init__()
        self.yolo = yolo_model
        self.hook = hook
    def forward(self, x):
        self.hook.features = None
        _ = self.yolo.model(x)
        return self.hook.features

feat_extractor = Waldo30SingleLayerExtractor(model_yolo, feature_hook).to(seade)

#4.2) Transformid (transforms)
map_transform = T.Compose([
    T.Resize((1024, 1024)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
drone_transform = T.Compose([
    T.Resize((640, 640)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

@torch.no_grad()
def extrakti_omadused(pil_img, transform):
    x = transform(pil_img).unsqueeze(0).to(seade)
    feats = feat_extractor(x)
    print(f"Feature shape: {feats.shape if feats is not None else 'None'}")
    return feats

#5)Soojuskaardi arvutamine (heatmap calculation)
def compute_heatmap_ncc(mapF, droneF):
    print(f"Algne mapF.shape={mapF.shape}, droneF.shape={droneF.shape}")
    target_h = mapF.shape[2] // 3
    target_w = mapF.shape[3] // 3
    print(f"Skaleerin droneF: ({droneF.shape[2]}, {droneF.shape[3]}) -> ({target_h}, {target_w})")
    droneF = F.interpolate(droneF, size=(target_h, target_w), mode='bilinear', align_corners=False)
    print(f"Pärast skaleerimist droneF.shape={droneF.shape}")
    cross_term = F.conv2d(mapF, droneF, bias=None, stride=1)
    norm_drone = torch.sqrt(torch.sum(droneF ** 2))
    map_sq = mapF * mapF
    ones_ker = torch.ones_like(droneF)
    sum_map_sq = F.conv2d(map_sq, ones_ker, bias=None, stride=1)
    norm_map = torch.sqrt(sum_map_sq + 1e-8)
    ncc = cross_term / (norm_drone * norm_map + 1e-8)
    return ncc.squeeze(0).squeeze(0).cpu().numpy()

#6)Katvusala arvutamine (bbox calculation)
def arvuta_bbox_fov(xLe, yLe, alt_sea, ground_offset=40.0, fov_deg=70.0):
    alt_p = alt_sea - ground_offset if alt_sea else 200.
    if alt_p < 0: alt_p = 50.
    rad = math.radians(fov_deg)
    laius = 2 * alt_p * math.tan(rad / 2) * 4
    pool = laius / 2
    return [xLe - pool, yLe - pool, xLe + pool, yLe + pool]

#7)Yaw korrigeerimine (yaw correction)
def paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll, flight_yaw=None):
    if yaw is None and flight_yaw is None:
        return droonPil
    if pitch is not None and abs(pitch + 90) < 10:
        angle = 360.0 - yaw if yaw is not None else 0.0
        if flight_yaw is not None:
            yaw_diff = abs((yaw - flight_yaw + 360) % 360)
            if 160 <= yaw_diff <= 200:
                print(f"Yaw-flip tuvastatud (gimbal: {yaw:.1f}, flight: {flight_yaw:.1f}) => flip +180°")
                angle = (angle + 180) % 360
        print(f"Keeran droonipilti {angle:.1f} kraadi (nadir, gimbal yaw)")
        return droonPil.rotate(angle, expand=True)
    elif flight_yaw is not None:
        angle = 360.0 - flight_yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (flight yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)
    elif yaw is not None:
        angle = 360.0 - yaw
        print(f"Keeran droonipilti {angle:.1f} kraadi (fallback gimbal yaw), pitch={pitch}")
        return droonPil.rotate(angle, expand=True)
    return droonPil

#8)Pixelist L-EST koordinaatideks (coordinate transformation)
def pixel2lest(bbox, w, h, col, row):
    xmin, ymin, xmax, ymax = bbox
    x = xmin + (xmax - xmin) * (col / float(w))
    y = ymin + (ymax - ymin) * (row / float(h))
    return x, y

#9)Drooni teekonna geodataframe (drone trajectory geodataframe)
def drooni_teekond_gdf(flight_data):
    geometry = [Point(d['x'], d['y']) for d in flight_data]
    ajad = [d['aeg'] for d in flight_data]
    gdf = gpd.GeoDataFrame({'aeg': ajad}, geometry=geometry, crs="EPSG:3301")
    return gdf

#10)Drooni teekond (drone trajectory)
def joonista_drooni_teekond(gdf, margin=2000):
    minx, miny, maxx, maxy = gdf.total_bounds  
    minx -= margin
    miny -= margin
    maxx += margin
    maxy += margin

    bbox = [minx, miny, maxx, maxy] 
    orto = wms_maaamet_taust(bbox, width=2000, height=2000)
    if orto is None:
        print("Ei saanud WMS -> joonistamine katkestatud.")
        return

    arr = np.array(orto)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(arr, extent=[minx, maxx, miny, maxy], origin='lower')  
    gdf.plot(ax=ax, marker='o', color='red', markersize=50, label='Droonipunktid')

    if len(gdf) > 1:
        line_geom = LineString(gdf.geometry.values)
        line_gdf = gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:3301")
        line_gdf.plot(ax=ax, color='blue', linewidth=2, label='Drooni teekond')

    for idx, row in gdf.iterrows():
        ax.text(row.geometry.x, row.geometry.y, str(idx+1), color='white', fontsize=12,
            ha='center', va='center', fontweight='bold')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_title("Drooni teekond (zoomitud, põhjasuunaline)")
    ax.legend()
    plt.show()

#11) Nihke arvutus (offset calculation in meters) ===
def arvuta_offset(x_est, y_est, x_gt, y_gt):
    return math.hypot(x_est - x_gt, y_est - y_gt)

#12) Täielik koodi väljakutsumine (full pipeline)
offsetid = []
kiirused = []

def full_pipeline(kaust, ground_offset=40.0):
    failid = [f for f in os.listdir(kaust) if f.lower().endswith(".jpg")]
    if not failid:
        print("Ei leidu droonifotosid:", kaust)
        return

    flight_data = []

    for fn in failid:
        path = os.path.join(kaust, fn)
        print(f"\nTöötlen faili: {path}")

        info = loe_exif_info(path)
        lat, lon = info['lat'], info['lon']
        alt = info['alt']
        fov = info['fov'] if info['fov'] else 70
        yaw, pitch, roll = info['gimbal_yaw'], info['gimbal_pitch'], info['gimbal_roll']

        if lat is None or lon is None:
            print("Puuduvad koordinaadid (lat/lon) -> fail vahele.")
            continue

        xLe, yLe = proj_3301(lat, lon)
        bbox = arvuta_bbox_fov(xLe, yLe, alt, ground_offset, fov)
        print("-> Arvutatud BBOX:", bbox)

        orto_pil = wms_maaamet_taust(bbox, width=1024, height=1024)
        if orto_pil is None:
            print("Ei õnnestunud WMS pildi laadimine -> fail vahele.")
            continue

        droonPil = Image.open(path).convert("RGB")
        droonPil = paranda_yaw_kui_nadir(droonPil, yaw, pitch, roll)
        droonPil = droonPil.resize((256, 256), Image.BILINEAR)

        #Vaade droonipildist ja ortokaardist (view of droneimage and ortophoto)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
        ax1.imshow(orto_pil)
        ax1.set_title("Ortofoto (1024x1024)")
        ax2.imshow(droonPil)
        ax2.set_title("Droonipilt (256x256, yaw korrigeeritud)")
        plt.show()

        start_time = time.time()
        mapF = extrakti_omadused(orto_pil, map_transform)
        droF = extrakti_omadused(droonPil, drone_transform)
        if mapF is None or droF is None:
            print("Feature ekstraktsioon ebaõnnestus -> fail vahele.")
            continue

        simmap = compute_heatmap_ncc(mapF, droF)
        best_i, best_j = np.unravel_index(simmap.argmax(), simmap.shape)
        best_val = simmap[best_i, best_j]
        print(f"Max sarnasus: {best_val:.4f} asukohas (row={best_i}, col={best_j})")

        end_time = time.time()
        kiirused.append(end_time - start_time)

        w, h = simmap.shape[1], simmap.shape[0]
        x_hit, y_hit = pixel2lest(bbox, w, h, best_j, best_i)
        flight_data.append({'aeg': 'unknown', 'x': x_hit, 'y': y_hit})

        plt.figure(figsize=(8, 6))
        plt.imshow(simmap, cmap='hot', interpolation='nearest', origin='upper')
        plt.plot([best_j], [best_i], 'ro')
        plt.colorbar(label='sarnasus')
        plt.title(f"{fn}: max={best_val:.4f}")
        plt.show()

        #Nihke arvutus (offset calculation)
        x_gt, y_gt = proj_3301(lat, lon)
        offset = arvuta_offset(x_hit, y_hit, x_gt, y_gt)
        offsetid.append(offset)
        print(f"Offset: {offset:.2f} m")

    if not flight_data:
        print("Pole lennupunkte -> pipeline lõpetatud.")
        return

    #Lennuteekond (flight trajectory)
    gdf = drooni_teekond_gdf(flight_data)
    joonista_drooni_teekond(gdf, margin=2000)

    offset_np = np.array(offsetid)
    kiir_np = np.array(kiirused)

    print("\n=== KOKKUVÕTE ===")
    print(f"Piltide arv: {len(offset_np)}")
    print(f"Keskmine offset: {offset_np.mean():.2f} m")
    print(f"Min offset: {offset_np.min():.2f} m")
    print(f"Max offset: {offset_np.max():.2f} m")
    print(f"Standardhälve: {offset_np.std():.2f} m")
    print(f"Keskmine töötlemisaeg: {kiir_np.mean()*1000:.1f} ms")

    tabel = pd.DataFrame({'Fail': failid, 'Offset_m': offset_np, 'Aeg_ms': kiir_np * 1000})
    print("\nTABEL:\n", tabel)

def naide_kasutus():
    kaust = "LISA KOODI RAKENDAMISEKS ENDA PILDIKAUST"
    full_pipeline(kaust, ground_offset=40)

naide_kasutus()
